In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# pip install -U scikit-learn

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

In [5]:
!pip install fasttext
import fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4395560 sha256=f41c20189236cd1eb0ad2d783feb58026f6f8df763d3b33d53af7e3b3532dbeb
  Stored in directory: /root/.cache/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [6]:
df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)
df.sample(5)

<ipython-input-6-7fde084fdf7a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('drive/MyDrive/TextAnalytics/preprocessed.csv',engine="python", encoding="utf-8", error_bad_lines=False)


,text,class,char_length,token_length
184825,play lost friend request play box smurf iidocp...,1,73,11
176319,update hole crush situation thing yeah managed...,0,168,25
174709,hope left barely feel human anymore think poss...,1,204,32
114127,handle anymore life 've having love 've felt '...,1,1020,158
141467,black baggy hoodies black track pant outfit fi...,0,173,26


In [7]:
model = fasttext.load_model('drive/MyDrive/TextAnalytics/cc.en.300.bin')
def get_text_vector(sentence): return model.get_sentence_vector(sentence)

In [8]:
df['text_vectors'] = df['text'].apply(get_text_vector)
df.sample(5)

,text,class,char_length,token_length,text_vectors
130315,finding condom,0,14,2,"[0.044181734, 0.0070814663, 0.020810774, 0.100..."
199682,mother beat cancer end outliving meet come imm...,1,1127,161,"[-0.009415357, 0.0016718886, 0.015485296, 0.07..."
32257,help need help like longest time 've numbed me...,1,359,57,"[-0.004437286, 0.0011709888, 0.02233081, 0.070..."
188325,item throwaway account way going straight poin...,1,270,44,"[0.0020955806, 0.013501251, 0.060259078, 0.087..."
121825,girl tell boy pissed pant thank attention,0,43,7,"[0.040485423, 0.030556604, 0.04785307, 0.06570..."


In [9]:
X, y = df['text_vectors'], df['class']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# **Model with default Hyperparameters**

In [11]:
## default model
clf=XGBClassifier(learning_rate=0.3, n_estimators=100, min_child_weight=1, max_depth=3, booster='gbtree', 
                  objective='binary:logistic', tree_method='gpu_hist')
#Creating the model on Training Data
XGP_model = clf.fit(list(X_train),y_train)

y_pred = XGP_model.predict(list(X_test))

#Confusion Matrix
clf_tn, clf_fp, clf_fn, clf_tp = confusion_matrix(y_test, y_pred).ravel()
#measurements
clf_accuracy = (clf_tp+clf_tn)/(clf_tp+clf_tn+clf_fp+clf_fn)
print(" Accuracy", round(clf_accuracy,4))
clf_precision = clf_tp/(clf_tp+clf_fp)
clf_recall = clf_tp/(clf_tp+clf_fn)
clf_f1_score = (2*clf_precision)*(clf_recall)/(clf_precision+clf_recall)
print(" Precision",round(clf_precision,4),"\n","Recall",round(clf_recall,4),"\n","F1",round(clf_f1_score,4))

 Accuracy 0.9043
 Precision 0.9082 
 Recall 0.9008 
 F1 0.9045


In [12]:
param_test1 = {
    'learning_rate': [0.1, 0.2, 0.3],
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2),
    'n_estimators': [50, 100, 150, 200, 500, 1000]
}

grid_search = GridSearchCV(estimator = XGBClassifier(objective= 'binary:logistic', tree_method='gpu_hist', booster='gbtree'), 
                            param_grid = param_test1, scoring='roc_auc', n_jobs=-1, cv=5)

In [ ]:
grid_search.fit(list(X_train),y_train)
grid_search.best_params_, grid_search.best_score_

# **Model with Hyperparemeter Tuning**

In [14]:
clf = XGBClassifier(learning_rate=grid_search_gbtree.best_params_['learning_rate'], n_estimators=grid_search_gbtree.best_params_['n_estimators'], 
                    min_child_weight=grid_search_gbtree.best_params_['min_child_weight'], max_depth=grid_search_gbtree.best_params_['max_depth'], 
                    booster='gbtree', 
                   objective='binary:logistic', tree_method='gpu_hist')
XGB = clf_gbtree.fit(list(X_train),y_train)

In [15]:
y_pred = XGB.predict(list(X_test))

#Confusion Matrix
clf_tn, clf_fp, clf_fn, clf_tp = confusion_matrix(y_test, y_pred).ravel()
#measurements
clf_accuracy = (clf_tp+clf_tn)/(clf_tp+clf_tn+clf_fp+clf_fn)
print(" Accuracy", round(clf_accuracy,4))
clf_precision = clf_tp/(clf_tp+clf_fp)
clf_recall = clf_tp/(clf_tp+clf_fn)
clf_f1_score = (2*clf_precision)*(clf_recall)/(clf_precision+clf_recall)
print(" Precision",round(clf_precision,4),"\n","Recall",round(clf_recall,4),"\n","F1",round(clf_f1_score,4))

 Accuracy 0.9223
 Precision 0.9263 
 Recall 0.9186 
 F1 0.9224
